## The Decision Problem

We make widgets. Have a set of production facilities that produce boxes of widgets. There is also a set of distribution locations that will then distribute the widgets for sale. Each distribution center has a forecasted demand and each production facility has a min and max number of widgets it can make during this period. We need to ensure that each distribution facility receives enough widgets to satisfy demand from production and we want to do this at minimal cost. The minimum production is 75% of the production facilities max value.

## Sets

Our sets are:
*   $P = \{`\textrm{Baltimore'}, `\textrm{Cleveland'}, `\textrm{Little Rock'}, `\textrm{Birmingham'}, `\textrm{Charleston'} \}$
*   $D = \{`\textrm{Columbia'}, `\textrm{Indianapolis'}, `\textrm{Lexington'}, `\textrm{Nashville'}, `\textrm{Richmond'}, `\textrm{St. Louis'} \}$



In [1]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 33.8 MB/s eta 0:00:00


In [2]:
# Import packages
import pandas as pd
import gurobipy as gb
from gurobipy import GRB

In [3]:
# Define sets P and D respectively
production = ['Baltimore', 'Cleveland', 'Little_Rock', 'Birmingham', 'Charleston']
distribution = ['Columbia', 'Indianapolis', 'Lexington', 'Nashville', 'Richmond', 'St_Louis']

In [4]:
# Define the gurobipy model for the decision problem
m = gb.Model('widgets')

Restricted license - for non-production use only - expires 2026-11-23


## Parameters

*   $m_{p}$ is the max production in location $p, \forall p \in P$ $\hspace{9cm} \texttt{max_prod[p]}$
*   $n_{d}$ is the number of customers for a distribution center $d, \forall d \in D$ $\hspace{5.5cm} \texttt{n_demand[d]}$
*   $c_{p,d}$ is the cost to ship a widget between location $p$ and location $d$, $\forall p \in P, d \in D$ $\hspace{2.4cm} \texttt{cost[p, d]}$

In [5]:
# Use squeeze=True to make the costs a series
# Downloaded from https://github.com/Gurobi/modeling-examples/tree/master
path = "drive/MyDrive/Colab Notebooks/gurobi_101/"
transp_cost = pd.read_csv(path + "cost.csv", index_col=[0, 1]).squeeze()

In [6]:
# Pivot to view the costs a bit easier
transp_cost.reset_index().pivot(index='production', columns='distribution', values='cost')

distribution,Columbia,Indianapolis,Lexington,Nashville,Richmond,St_Louis
production,,,,,,
Baltimore,4.50,5.09,4.33,5.96,1.96,7.30
Birmingham,3.33,4.33,3.38,1.53,5.95,4.01
Charleston,3.02,2.61,1.61,4.44,2.36,4.60
Cleveland,2.43,2.37,2.54,4.13,3.20,4.88
Little_Rock,6.42,4.83,3.39,4.40,7.44,2.92


In [7]:
max_prod = pd.Series([180, 200, 140, 80, 180], index=production, name="max_production")
n_demand = pd.Series([89, 95, 121, 101, 116, 181], index=distribution, name="demand")

In [8]:
max_prod.to_frame()

,max_production
Baltimore,180
Cleveland,200
Little_Rock,140
Birmingham,80
Charleston,180


In [9]:
n_demand.to_frame()

,demand
Columbia,89
Indianapolis,95
Lexington,121
Nashville,101
Richmond,116
St_Louis,181


We also have the requirement that each production facility needs to produce at 75% of this maximum output. We'll denote this value by $a$ in the formulation "frac" for the fraction of maximum production required. Initially we set $a = 0.75$.

In [10]:
frac = 0.75

## Decision Variables
The decision here is to determine the number of boxes to send from each production facility to each distribution location.

Let $x_{p, d}$ be the number of widgets that are produced at facility $p$ and shipped to location $d$.

## Add Variables in gurobipy
$\texttt{gurobipy}$ let's you add decision variables primarily with two (similar) commands:

*   $\texttt{addVar()}$ adds a single variable
*   $\texttt{addVars()}$ adds a group of variables by sets/indices

In [11]:
# Loop through each p and d combination to create a decision variable
x = {}
for p in production:
    for d in distribution:
        x[p, d] = m.addVar(name = p+"_to_"+d)
m.update() # Updates the model to include any changes that have been made

In [12]:
x

{('Baltimore', 'Columbia'): <gurobi.Var Baltimore_to_Columbia>,
 ('Baltimore', 'Indianapolis'): <gurobi.Var Baltimore_to_Indianapolis>,
 ('Baltimore', 'Lexington'): <gurobi.Var Baltimore_to_Lexington>,
 ('Baltimore', 'Nashville'): <gurobi.Var Baltimore_to_Nashville>,
 ('Baltimore', 'Richmond'): <gurobi.Var Baltimore_to_Richmond>,
 ('Baltimore', 'St_Louis'): <gurobi.Var Baltimore_to_St_Louis>,
 ('Cleveland', 'Columbia'): <gurobi.Var Cleveland_to_Columbia>,
 ('Cleveland', 'Indianapolis'): <gurobi.Var Cleveland_to_Indianapolis>,
 ('Cleveland', 'Lexington'): <gurobi.Var Cleveland_to_Lexington>,
 ('Cleveland', 'Nashville'): <gurobi.Var Cleveland_to_Nashville>,
 ('Cleveland', 'Richmond'): <gurobi.Var Cleveland_to_Richmond>,
 ('Cleveland', 'St_Louis'): <gurobi.Var Cleveland_to_St_Louis>,
 ('Little_Rock', 'Columbia'): <gurobi.Var Little_Rock_to_Columbia>,
 ('Little_Rock', 'Indianapolis'): <gurobi.Var Little_Rock_to_Indianapolis>,
 ('Little_Rock', 'Lexington'): <gurobi.Var Little_Rock_to_Lexing

In [ ]:
# Alternate way to specify variables
# x = m.addVars(production, distribution, name="prod_ship")
# m.update()

In [ ]:
# Yet another way to specify variables
# x = m.addVars(transp_cost.index, name="prod_ship")
# m.update()

## Constraints
Adding constraints to a model is similar to adding variables

*   $\texttt{addConstr()}$ adds a single constraint
*   $\texttt{addConstrs()}$ adds a group of constraints using a Python generator expression

We'll formulate the demand constraints for each distribution location first and add them to the model

$$
\sum_{p} x_{p,d} \ge n_{d}, \hspace{1cm} \forall\, d \in D \hspace{1cm} \texttt{meet_demand[d]}
$$

This will be the first time we use $\texttt{gb.quicksum()}$. There are other ways to sum expressions in gurobipy and while this method isn't the most concise to code, it is easy to compare it to the summation in the formulation to see how it works.


In [13]:
# Adding all the demand constraints
meet_demand = m.addConstrs((gb.quicksum(x[p, d] for p in production) >= n_demand[d] for d in distribution),
                           name = "meet_demand")
m.update()

In [14]:
meet_demand

{'Columbia': <gurobi.Constr meet_demand[Columbia]>,
 'Indianapolis': <gurobi.Constr meet_demand[Indianapolis]>,
 'Lexington': <gurobi.Constr meet_demand[Lexington]>,
 'Nashville': <gurobi.Constr meet_demand[Nashville]>,
 'Richmond': <gurobi.Constr meet_demand[Richmond]>,
 'St_Louis': <gurobi.Constr meet_demand[St_Louis]>}

Next we have the maximum number of widgets each production facility can make. We also have that each facility must make at least 75% of its max production.

$$
\begin{align}
\sum_{d} x_{p, d} \leq m_{p}, \hspace{1.9cm} &\forall\, p \in P \hspace{1cm} \texttt{can_produce[p]}\\
\sum_{d} x_{p, d} \geq a \times m_{p}, \hspace{1cm} &\forall\, p \in P \hspace{1cm} \texttt{must_produce[p]}
\end{align}
$$

In [15]:
# Adding all production constraints
can_produce = m.addConstrs((gb.quicksum(x[p, d] for d in distribution) <= max_prod[p] for p in production),
                           name = "can_produce")
must_produce = m.addConstrs((gb.quicksum(x[p, d] for d in distribution) >= frac * max_prod[p] for p in production),
                             name = "must_produce")
m.update()

In [16]:
can_produce

{'Baltimore': <gurobi.Constr can_produce[Baltimore]>,
 'Cleveland': <gurobi.Constr can_produce[Cleveland]>,
 'Little_Rock': <gurobi.Constr can_produce[Little_Rock]>,
 'Birmingham': <gurobi.Constr can_produce[Birmingham]>,
 'Charleston': <gurobi.Constr can_produce[Charleston]>}

## Objective Function
This is done using `setObjective()`. The second argument (in this case
`GRB.MINIMIZE`) is called the model's *sense*. For maximization problem we would use `GRB.MAXIMIZE`.

$$
\textrm{minimize} \sum_{p, d} c_{p, d} x_{p, d} \hspace{1cm} \forall\, p \in P, d \in D
$$



In [17]:
# Set objective to minimize the cost of transportation
m.setObjective(gb.quicksum(transp_cost[p, d] * x[p, d] for p in production for d in distribution),
               GRB.MINIMIZE)
m.update()

## Find, Extract, and Analyze the Solution
Before running the optimization, it is a good idea to write an lp file. This is a text file that prints out the variables, constraints, and objective like we would see in the *formulation*, just without the summation symbols and using the names we designated

In [18]:
m.write(path + 'widget_shipment.lp')

In [19]:
# Run the Optimization
m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 16 rows, 30 columns and 90 nonzeros
Model fingerprint: 0x20186c14
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 7e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 2e+02]
Presolve removed 5 rows and 0 columns
Presolve time: 0.01s
Presolved: 11 rows, 35 columns, 65 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.610000e+02   0.000000e+00      0s
      15    1.7048900e+03   0.000000e+00   0.000000e+00      0s

Solved in 15 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.704890000e+03


## Extract the Solution
There are many ways to get the values of decision variables out of gurobipy

In [ ]:
x_values = pd.Series(m.getAttr('X', x), name="shipment", index=transp_cost.index)
sol = pd.concat([transp_cost, x_values], axis=1)
sol

cost  shipment
production  distribution                
Baltimore   Columbia      4.50       0.0
            Indianapolis  5.09       0.0
            Lexington     4.33       0.0
            Nashville     5.96      19.0
            Richmond      1.96     116.0
            St_Louis      7.30       0.0
Cleveland   Columbia      2.43      89.0
            Indianapolis  2.37      95.0
            Lexington     2.54       0.0
            Nashville     4.13       2.0
            Richmond      3.20       0.0
            St_Louis      4.88       0.0
Little_Rock Columbia      6.42       0.0
            Indianapolis  4.83       0.0
            Lexington     3.39       0.0
            Nashville     4.40       0.0
            Richmond      7.44       0.0
            St_Louis      2.92     140.0
Birmingham  Columbia      3.33       0.0
            Indianapolis  4.33       0.0
            Lexington     3.38       0.0
            Nashville     1.53      80.0
            Richmond      5.95       0.0
            St_Louis      4.01       0.0
Charleston  Columbia      3.02       0.0
            Indianapolis  2.61       0.0
            Lexington     1.61     121.0
            Nashville     4.44       0.0
            Richmond      2.36       0.0
            St_Louis      4.60      41.0

In [ ]:
sol[sol.shipment > 0]

cost  shipment
production  distribution                
Baltimore   Nashville     5.96      19.0
            Richmond      1.96     116.0
Cleveland   Columbia      2.43      89.0
            Indianapolis  2.37      95.0
            Nashville     4.13       2.0
Little_Rock St_Louis      2.92     140.0
Birmingham  Nashville     1.53      80.0
Charleston  Lexington     1.61     121.0
            St_Louis      4.60      41.0

In [ ]:
# You can the name and value of all the decision variables:
all_vars = {v.varName: v.x for v in m.getVars()}
all_vars

{'Baltimore_to_Columbia': 0.0,
 'Baltimore_to_Indianapolis': 0.0,
 'Baltimore_to_Lexington': 0.0,
 'Baltimore_to_Nashville': 19.0,
 'Baltimore_to_Richmond': 116.0,
 'Baltimore_to_St_Louis': 0.0,
 'Cleveland_to_Columbia': 89.0,
 'Cleveland_to_Indianapolis': 95.0,
 'Cleveland_to_Lexington': 0.0,
 'Cleveland_to_Nashville': 2.0,
 'Cleveland_to_Richmond': 0.0,
 'Cleveland_to_St_Louis': 0.0,
 'Little_Rock_to_Columbia': 0.0,
 'Little_Rock_to_Indianapolis': 0.0,
 'Little_Rock_to_Lexington': 0.0,
 'Little_Rock_to_Nashville': 0.0,
 'Little_Rock_to_Richmond': 0.0,
 'Little_Rock_to_St_Louis': 140.0,
 'Birmingham_to_Columbia': 0.0,
 'Birmingham_to_Indianapolis': 0.0,
 'Birmingham_to_Lexington': 0.0,
 'Birmingham_to_Nashville': 80.0,
 'Birmingham_to_Richmond': 0.0,
 'Birmingham_to_St_Louis': 0.0,
 'Charleston_to_Columbia': 0.0,
 'Charleston_to_Indianapolis': 0.0,
 'Charleston_to_Lexington': 121.0,
 'Charleston_to_Nashville': 0.0,
 'Charleston_to_Richmond': 0.0,
 'Charleston_to_St_Louis': 41.0}

In [ ]:
# Or we can interate over a specific variable and only return values that are of interest to us.
xvals = {k: v.x for k, v in x.items() if v.x > 0}
xvals

{('Baltimore', 'Nashville'): 19.0,
 ('Baltimore', 'Richmond'): 116.0,
 ('Cleveland', 'Columbia'): 89.0,
 ('Cleveland', 'Indianapolis'): 95.0,
 ('Cleveland', 'Nashville'): 2.0,
 ('Little_Rock', 'St_Louis'): 140.0,
 ('Birmingham', 'Nashville'): 80.0,
 ('Charleston', 'Lexington'): 121.0,
 ('Charleston', 'St_Louis'): 41.0}